In [ ]:


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df_predict = pd.read_csv('/content/drive/MyDrive/Data Science/2023 Cars/2023Cars_no_price.csv')

df_predict.columns

#We rename the columns because some, become clumsy to work with, especially with all those spaces
column_name_changes = {
    ' Price ($) ': 'Price',
    ' Exterior Features                                           ': 'Exterior Features',
    ' Interior Features                                ': 'Interior Features',
    ' Car Make ': 'Car Make',
    ' Car Model   ': 'Car Model',
    ' Year ': 'Year',
    ' Body Type ': 'Body Type',
    ' Color Options                 ': 'Color Options',
    ' Fuel Type ': 'Fuel Type',
    'Engine Size (L)': 'Engine Size(L)',
    ' Horsepower ' : 'Horsepower',
    ' Torque (Nm) ' : 'Torque (Nm)',
    ' Transmission Type ': 'Transmission Type',
    ' Acceleration (0-60 mph) ': 'Acceleration (0-60 mph)',
    ' Top Speed (mph) ': 'Top Speed (mph)',
    ' Mileage (MPG) ': 'Mileage (MPG)',
    ' Safety Features                                      ': 'Safety Features',
    ' Entertainment Features               ': 'Entertainment Features',
    ' Customer Ratings ': 'Customer Ratings',
    ' Sales Figures (Units Sold)': 'Sales Figures (Units Sold)'

}

df_predict.rename(columns=column_name_changes, inplace=True)


#We also drop the comma and convert into float
df_predict['Sales Figures (Units Sold)'] = df_predict['Sales Figures (Units Sold)'].str.replace(',', '').astype(float)

#In this one, we drop the 'out of five (x/5)' part of the rating, might change the columns name to better reflect this in the future
df_predict['Customer Ratings'] = df_predict['Customer Ratings'].str.split('/').str[0]

#We drop the word seconds from the column and convert it to a float
df_predict['Acceleration (0-60 mph)'] = df_predict['Acceleration (0-60 mph)'].str.replace(' seconds', '').astype(float)

#The following was to drop an abreviation that appeared as (est.) when it came to electric cars
df_predict['Horsepower'] = df_predict['Horsepower'].str.split('(').str[0]
df_predict['Torque (Nm)'] = df_predict['Torque (Nm)'].str.split('(').str[0]
df_predict['Top Speed (mph)'] = df_predict['Top Speed (mph)'].str.split('(').str[0]
df_predict['Mileage (MPG)'] = df_predict['Mileage (MPG)'].str.split('(').str[0]

#After seeing the previous graph, we could see something was a miss, so after further diggin, we found 2 entries that were wrong, so we went ahead and fix them
#Here we modify row 103, and 139, after some digging, we found that the info in some rows, were actually in another one, so, we set out to put everything in its place
rows_to_modify = [80, 147]
columns_to_modify = ['Customer Ratings', 'Sales Figures (Units Sold)', 'Mileage (MPG)', 'Entertainment Features', 'Safety Features']
new_values = [4.9, 900, 110, '10.2-inch Infotainment Display', 'ABS, Airbags, HD Rear Vision Camera, Lane Keep Assist with Lane Departure Warning']

for row_index in rows_to_modify:
    for col_index, column in enumerate(columns_to_modify):
        new_value = new_values[col_index]
        df_predict.loc[row_index, column] = new_value


rows_to_modify = [80]
columns_to_modify = ['Exterior Features', 'Interior Features']
new_values = ['LED Headlights, Roof Rails, 17-inch Alloy Wheels', 'HD Rear Wireless Smartphone Charging']

for row_index in rows_to_modify:
    for col_index, column in enumerate(columns_to_modify):
        new_value = new_values[col_index]
        df_predict.loc[row_index, column] = new_value


rows_to_modify = [147]
columns_to_modify = ['Interior Features']
new_values = ['Leather Seats, Keyless Entry, Wireless Smartphone Charging']

for row_index in rows_to_modify:
    for col_index, column in enumerate(columns_to_modify):
        new_value = new_values[col_index]
        df_predict.loc[row_index, column] = new_value

#We convert Top Speed to a float
df_predict['Top Speed (mph)'] = df_predict['Top Speed (mph)'].str.replace(' mph', '').astype(float)

#Convert Mileage to float
df_predict['Mileage (MPG)'] = df_predict['Mileage (MPG)'].str.replace(' MPG', '').astype(float)


#We convert Torque to float and drop the empty values of some rows
df_predict['Torque (Nm)'] = df_predict['Torque (Nm)'].str.replace(' N/A         ', '').str.replace(' Nm', '').str.strip()
df_predict = df_predict[df_predict['Torque (Nm)'] != '']  # Remove rows with empty values
df_predict['Torque (Nm)'] = df_predict['Torque (Nm)'].astype(float)

#Convert Horsepower into float
df_predict['Horsepower'] = df_predict['Horsepower'].astype(float)

#Once found, we can see its actually not a lot of values, so it makes no harm if we drop em for a cleaner workspace
df_predict = df_predict.drop(df_predict.loc[df_predict['Mileage (MPG)'].isnull()].index)
df_predict = df_predict.drop(df_predict.loc[df_predict['Exterior Features'].isnull()].index)
df_predict = df_predict.drop(df_predict.loc[df_predict['Interior Features'].isnull()].index)

#We dropped the outliers

rows_to_drop = [148, 40, 107,135,41,88,4]
df_predict = df_predict.drop(rows_to_drop)

#Log transform for slighly right skewness
df_predict['NormMileage'] = np.log(df_predict['Mileage (MPG)'])

#Exponential transformation (X^2) for slightly right skewness
df_predict['NormAcceleration'] = np.power(df_predict['Acceleration (0-60 mph)'], 2)


#Log transformation for right skewness
df_predict['NormTorque'] = np.log(df_predict['Torque (Nm)'])


#Log transformation for left skewness
df_predict['NormHorsepower'] = np.log(df_predict['Horsepower'])


#We import our previouslt trained model
import catboost
from catboost import CatBoostRegressor

catboost_model = CatBoostRegressor()
catboost_model.load_model('/content/drive/MyDrive/Data Science/2023 Cars/carboost_model.cbm')


<ipython-input-3-ed27150875e4>:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict['Torque (Nm)'] = df_predict['Torque (Nm)'].astype(float)
<ipython-input-3-ed27150875e4>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict['Horsepower'] = df_predict['Horsepower'].astype(float)
